<h2>Introduction</h2>

My family owns some commercial real estate in the Pilsen neighborhood of Chicago. We are considering selling these properties to purchase property in another neighborhood or city.

Extrapolating based only on <i>price</i> over <i>time</i> might be sufficient for profitable real estate investment at scale. But for an individual with a small, non-diverse property portfolio, for whom the value of a given property is a significant contributor to net worth and for whom extracted rents account for a significant portion, if not the sole source of income, this approach is little more than a gamble. Therefore, a more informed model is warranted.

Models predicting real estate prices based on variables besides <i>time</i> are not new (see Zillow, for example), but-
- The inclusion of specific location features (e.g. nearby venues) in these models is limited. 
- For residential real estate, the weights of property features (e.g. "number of bedrooms") can be expected to be greater than for commercial real estate; i.e. a great house doesn't necessarily make for a great place of business.

Even then I'm sure models exist doing what I'm setting out to do here, but oh well, I'm not at a point where I'm going to be doing any groundbreaking work anytime soon.

Using this model, we should be able to make predictions based on speculative developments (e.g. if we learn of plans to open a new museum in the area) and while one development might not have a significant impact, we could populate the model with many and evaluate their cumulative effects. Sources of this information would be city council agenda items, talks with aldermen, etc, but would be outside the scope of my aim for this project.

<h2>Data</h2>

The variable we are interested in predicting is comemrcial real estate prices

In the first iteration of this model, to make these predictions, I employ:
- Chicago GeoJSON data
- Chicago POI data
- Chicago commercial real estate prices

The GeoJSON data is used to establish the boundaries within which points of interest and real estate prices are considered. This data is made available by the City of Chicago, and has been structured for use by Github user <i>wbdean</i> available at the following URL: https://raw.githubusercontent.com/wbdean/chicago_geojson/master/chicagoCommunities.geojson

POI data is sourced from the Foursquare API, for which I obtained approximately 17,000 venues' data, of which approximately 14,000 were used.

Commercial real estate prices are sourced from the ATTOM Property API, information about which you will find here: https://www.attomdata.com/solutions/api/. Sale amount data is very sparse. In place of sale amounts, for purposes of building the model, I instead use the sum of the assessed improvement, land, and title values, divided by 25% to account for the difference between assessed and fair market values, as per this handbook on the Cook County assessment process: https://www.civicfed.org/sites/default/files/100405_CookCountyAssessmentPrimer.pdf. Though this source is 9 years old, this assessed value to fair market value ratio seems to fit the data I collected based on a comparison of the estimated value using this ratio and the actual sale value.

Future iterations of this model should include:
- Time series data - for example: the effect of the addition of a new POI, even if significant, may not be immediate
- Demographic data - my model assumes demographics, specifically income, are approximated, albeit roughly, by points of interest; for example: many fast food restaurants and few fine dining restaurants would indicate lower income and, in turn, lower property prices

I have chosen to exclude them for the time being as:
- Time series data, particularly with regards to points of interest, will present a significant challenge for data collection
- Without time series data, demographic data, which is collected only in long intervals, loses much of its predictive value
- Pricing of special property types (e.g. billboards), while of interest to owners of such properties, would require domain knowledge which I do not have - for the time being, in my model these should be effectively controlled for in my real estate price calculation's removal of outliers

<h2>Methodology</h2>

The basic structure of this analysis consists of:
1. Clustering communities based on points of interest
2. Binning communities based on commercial real estate pricing
3. Evaluating the similarity between the community POI clusters and pricing bins

<h3>Clustering communities based on points of interest</h3>

I began by loading the GeoJSON data and fitting it into a dataframe containing the community and its geometry, with the intent to run a Foursquare API query on each of these communities. However, I ran into 2 challenges:
1. Foursquare does not allow you to query by polygonal geometry
2. Foursquare's results are limited when querying a large bounding box

To overcome these limitation, I created a bounding box for the whole of Chicago and divided it into a 200x200 grid, resulting in 40,000 roughly 230 meter by 220 meter boxes. To reduce the number of queries being run and improve performance, I then calculated which of these boxes had at least one of their corners intersect with the aforementioned community geometry, which left me with 17,176 boxes and thus 17,176 queries to run.

Results of these queries are as follows:
- 6,061 returned results
- 11,115 returned no results
- 17,024 venues' details were returned
- 14,279 venues' details included full addresses

Of the 14,279 venues with addresses, 13,781 fit within the community boundaries, indicating that the "low resolution" 200x200 grid was effective in achieving the desired result, with few extraneous venues. These venues' locations are shown below:

<img src='https://i.postimg.cc/bw0mjxZM/20190428214828.png' border='0'></a>

Next, I calculated the frequency of different venues within each community, resulting in the following dataframe:

<img src='https://i.postimg.cc/yV8RKMrj/20190428215524.png' border='0'/></a>

Finally, I clustered Chicago's communities for k=2 through k=10, resulting in the following for example:

<img src='https://i.postimg.cc/52SNF27G/20190428220115.png' border='0'/></a>

Other clusterings below:
- https://i.postimg.cc/RNSmTwsY/20190428220104.png
- https://i.postimg.cc/GHXCg96m/20190428220109.png
- https://i.postimg.cc/BPCh6L2L/20190428220112.png
- https://i.postimg.cc/grPmnCvm/20190428220118.png
- https://i.postimg.cc/G9JrYXQY/20190428220121.png
- https://i.postimg.cc/PCckh9XG/20190428220124.png
- https://i.postimg.cc/yJZCNjxD/20190428220127.png

<h3>Binning communities based on commercial real estate pricing</h3>

I first attempted to scrape websites like Century21 to retrieve real estate pricing data, but the resulting dataset was much too sparse to work with. Next I turned to the existing venue data.

I filtered the venue data down to a single category to limit possible influence of inconsistent building features. I chose restaurants, in particular, because restaurants have consistent infrastructure requirements, thus further limiting the effect of building features on price. This returned 2,923 results.

After filtering, I found that three communities, Riverdale, Burnside, and Oakland, did not have any restaurant venues within them. I checked their full venue list from the Foursquare queries and found that they had limited results already, and it would not be possible to perfectly accommodate them.

Next I calculated the distance between each point of interest and the center of each bounded section of each community. This allowed me to evaluate communities using more than just the POIs contained within them, as in cases where the community does not contain data of any such venues and when the real estate data returned was sparse, as I expected given my earlier attempt to retrieve this data. From these, I filtered these results to the top 5 nearest restaurants to each community's center point and pulled their property details from the ATTOM API.

On average, the top 5 resulted in fewer than one successful result per community, so ran it again iteratively over a few days to pull as much data as I could within the query limits of my account. I repeated this process up through the nearest 30 restaurants.

The results are as follows:
- 1,357 results obtained
- 694 results contained complete property assessment data
- 375 results included building square footage
- 200 results included the last sale price

I processed these results by:
- Calculating a fair market value as 4x the sum of the improvement, land, and title assessments
- Filtered out properties with outlying square footage and acreage
- From the remaining properties, divided the fair market price per square foot and price per acre

Then I grouped these results by community. The summary statistics are as follows:

<img src='https://i.postimg.cc/yNjVd66P/20190428230143.png' border='0'/></a>

And below is a section of the resulting dataframe:

<img src='https://i.postimg.cc/mDy3r5tM/20190428232805.png' border='0'/></a>

There are at least a couple of inaccuracies here I am aware of from my own experience with this market. Douglas looks extremely suspicious, and after investigating, I found it is definitely not representative of the actual community, but expanding the scope of the nearest venues would adversely affect the data quality for other communities. 

I elected to proceed anyway because most appear more-or-less in-line with expectations and these abnormalities should settle themselves when combined into their respective clusters. I would, however, like to revisit this at a later date with a more complete dataset and/or a model better able to accommodate the missing data.

This leaves a range of 785.79, which appears to be skewed right:

<img src='https://i.postimg.cc/5NnKxprD/20190429000706.png' border='0'/></a>

However, with an IQR of 114.77, using the rule of thumb of outliers being the median +/- 1.5 times the IQR, dropping the values outside this range leaves us with a range of 344.31, between 70.64 and 414.95. This gives us the following, much prettier distribution:

<img src='https://i.postimg.cc/YC9x3W3F/20190429003330.png' border='0'/></a>

(alright it's actually a fair bit uglier, but it gives us a better idea of how to bin these bad boys at least)

To calculate the bins, I divided the median +/- 1.5 times the IQR into the desired number, such that for 4 bins:
- Bin 1 is 70.64-156.72
- Bin 2 is 156.72-242.80
- Bin 3 is 242.80-328.88
- Bin 4 is 328.88-785.79

And so on.

This results in a dataframe like this (for 8 bins, 0-indexed):

<img src='https://i.postimg.cc/pLXy1J8k/20190429031759.png' border='0'/></a>

<h3>Evaluating the similarity between the community POI clusters and pricing bins</h3>

I had constructed a dictionary of dataframes of clustered communities for k=1 through k=10 and a dictionary of dataframes of binned communities for b=1 through b=10. I combined these, keying on the community name, to produce a dataframe comprised of the community, the pricing bin labels, and the cluster labels for each number of clusters/bins.

I increased these to 50 just to take a peek at what happened.

Using the scikitlearn homogeneity, completeness, and v_measure functions to evaluate similarities between the two sets of labels, I came to the following, less-than-stellar results. Here are 1-15:

<img src='https://i.postimg.cc/8z8pW8B2/20190429045701.png' border='0'/></a>

- Column 1 is the homogeneity score
- Column 2 is the completeness score
- Column 3 is the v_measure

For further reading on these: https://scikit-learn.org/stable/modules/clustering.html#homogeneity-completeness-and-v-measure

<h2>Results</h2>

Overall this model, in its current state, is not capable of any reliable estimation of pricing category based on neighborhood features. Overlap between the cluster labels and bin labels is limited, and while it increases with the k and b values, this is intuitive in the same way that a single bin is equal to a single cluster.

There is a small peak between the k and b values of 6 and 8 which coincides with an optimal number of clusters. The steady increase from 10 onward, to my understanding, can be safely discarded.

<h2>Discussion</h2>

Despite the less-than-promising results from this first iteration of this model, a few interesting observations came to light and would be worth investigating further. Most notably: The cluster maps in the areas of k=6-k=8 appear to be good approximations of certain demographic characteristics. For example, in this map with K = 8, there is an extremely strong relationship between the light-yellow cluster and the >80% black communities colored in the darker blue in the demographic map. It also identified Armour Square, highlighted in red on my map, as quite unique, which is fitting as it is the only majority Asian community (and home to a lot of delicious food).

<img src='https://i.postimg.cc/xjBdw7Ns/20190428220121.png' border='0'/></a><img src='http://www.gridgit.com/postpic/2013/02/chicago-racial-demographics-map_1394572.png' border='0'/></a>

(source of the demographics image: http://www.gridgit.com/post_city-of-chicago-neighborhood-demographics_1394570/)

It is also not a bad approximation of median household income, though it is not as immediately apparent.

This gives me confidence that there is merit in pursuing this further, but it will be no small feat. Challenges to be overcome include:
- Having more complete property data
- Constructing a better model for dealing with missing data
- Including time series data in the analysis
- Including demographic data in the analysis
- Including different venue types in the analysis
- Determining the relative weights of these other variables

In any case, it's a start.

<h3>Conclusion</h3>

In conducting this analysis, I found that the quality of information available for evaluating commercial real estate in such a way is far behind the quality of information available for residential real estate, at least for what is readily accessible to the layman. Comparing that with Zillow, that suggests that there remains a good market opportunity for an enterprising data scientist to carry this forward.